참고 : https://www.cahoover.com/technical-marketing/scraping-tripadvisor-reviews-using-python/

In [1]:
from selenium import webdriver
import time
import random
from urllib.request import urlopen
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import urllib.parse
from bs4 import BeautifulSoup as bs
import csv, re, requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# start scraping

1. url 들어가서
2. 데이터 추출하고
3. next page 누르고
4. 데이터 추출하고
5. next page 가 없다면 멈추기

In [2]:
hotel_list = pd.read_csv("일반호텔리스트2.csv", encoding = 'utf-8-sig', index_col=0)
hotel_list = hotel_list[hotel_list['호텔등급'] == 5]
hotel_list

,인증,등급,지역,호텔이름,호텔등급,url
32,greenglobe,platinum,Middle East,Fraser Suites Diplomatic Area - Bahrain,5.0,https://www.tripadvisor.com/Hotel_Review-g2939...
33,greenglobe,platinum,Middle East,Sofitel Dubai the Obelisk,5.0,https://www.tripadvisor.com/Hotel_Review-g2954...
34,greenglobe,platinum,Europe,JW Marriott Cannes,5.0,https://www.tripadvisor.com/Hotel_Review-g1872...
35,greenglobe,platinum,Europe,NH Collection Amsterdam Doelen,5.0,https://www.tripadvisor.com/Hotel_Review-g1885...
36,Travelife,award,Europe,Elegance Hotels International Marmaris,5.0,https://www.tripadvisor.com/Hotel_Review-g2980...
37,Travelife,award,Europe,Club Tuana Fethiye,5.0,https://www.tripadvisor.com/Hotel_Review-g2980...
38,Travelife,award,Europe,Aydinbey King's Palace & Spa,5.0,https://www.tripadvisor.com/Hotel_Review-g2979...
39,Travelife,award,Europe,GENNADI GRAND RESORT,5.0,https://www.tripadvisor.com/Hotel_Review-g1189...
40,greenglobe,eco,Europe,Hotel d'Angleterre,5.0,https://www.tripadvisor.com/Hotel_Review-g1895...


In [3]:
url_list = list(hotel_list['url'])
url_list = url_list[:]
len(url_list), url_list

(9,
 ['https://www.tripadvisor.com/Hotel_Review-g293997-d7697201-Reviews-Fraser_Suites_Diplomatic_Area_Bahrain-Manama.html',
  'https://www.tripadvisor.com/Hotel_Review-g295424-d21321279-Reviews-Sofitel_Dubai_the_Obelisk-Dubai_Emirate_of_Dubai.html',
  'https://www.tripadvisor.com/Hotel_Review-g187221-d197829-Reviews-JW_Marriott_Cannes-Cannes_French_Riviera_Cote_d_Azur_Provence_Alpes_Cote_d_Azur.html',
  'https://www.tripadvisor.com/Hotel_Review-g188590-d229171-Reviews-NH_Collection_Amsterdam_Doelen-Amsterdam_North_Holland_Province.html?spAttributionToken=MTgwMDU1MzA',
  'https://www.tripadvisor.com/Hotel_Review-g298033-d300033-Reviews-Elegance_Hotels_International_Marmaris-Marmaris_Marmaris_District_Mugla_Province_Turkis.html',
  'https://www.tripadvisor.com/Hotel_Review-g298031-d553651-Reviews-Club_Tuana_Fethiye-Fethiye_Mugla_Province_Turkish_Aegean_Coast.html',
  'https://www.tripadvisor.com/Hotel_Review-g297968-d1214278-Reviews-Aydinbey_King_s_Palace_Spa-Side_Manavgat_Turkish_Medit

In [4]:
hotel_name =list(hotel_list['호텔이름'])
hotel_name = hotel_name[:]
print(len(hotel_name)) 
hotel_name 

9


['Fraser Suites Diplomatic Area - Bahrain',
 'Sofitel Dubai the Obelisk',
 'JW Marriott Cannes',
 'NH Collection Amsterdam Doelen',
 'Elegance Hotels International Marmaris',
 'Club Tuana Fethiye',
 "Aydinbey King's Palace & Spa",
 'GENNADI GRAND RESORT',
 "Hotel d'Angleterre"]

In [28]:
#수집할 데이터
hotel_name_list = []
date_list=[] #리뷰가 쓰여진 날짜
rating_list=[] #해당 리뷰 별점
title_list=[] #해당 리뷰 제목
review_list=[] #해당 리뷰 본문
stay_day_list=[] #해당 리뷰 본문
trip_type_list=[] #여행 형태 ex) trveled with family
user_name_list=[]

In [29]:
for num, url in enumerate(url_list):
    time.sleep(3)
    driver = webdriver.Chrome('chromedriver.exe')
    urll = url
    driver.get(urll)
    
    page = 0 
    print(hotel_name[num], "시작")
    
    while True :
        page += 1
        if page % 10 == 0 :
            print(page,"번째 page 수집중")
        time.sleep(5) 
        driver.find_element_by_xpath(".//div[contains(@data-test-target, 'expand-review')]").click()
        container = driver.find_elements_by_xpath("//div[@data-reviewid]")
        dates = driver.find_elements_by_xpath(".//div[@class='cRVSd']")
        stay_days = driver.find_elements_by_xpath(".//span[@class='teHYY _R Me S4 H3']")
        test = driver.find_elements_by_xpath(".//div[@class='vTVDc']")

        for j in range(len(container)):
            try:
                rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                rating_list.append(rating)

                title = container[j].find_element_by_xpath(".//div[contains(@data-test-target, 'review-title')]").text
                title_list.append(title)

                review = container[j].find_element_by_xpath(".//q[@class='QewHA H4 _a']").text.replace("\n", "  ")
                review_list.append(review)

                date = " ".join(dates[j].text.split(" ")[-2:])
                date_list.append(date)

                user_name = " ".join(dates[j].text.split(" ")[:1])
                user_name_list.append(user_name)
                
                hotel_name_list.append(hotel_name[num]) 
                
                stay_day = " ".join(stay_days[j].text.split(" ")[-2:])
                stay_day_list.append(stay_day)

            except:
                pass
            
        for i in test:
            if "Trip type:" in i.text:
                x = i.text.find("Trip type:")
                trip_type_list.append(i.text[x:x+31])
            else :
                trip_type_list.append("None")
    
        try :
            driver.find_element_by_xpath('.//a[@class="ui_button nav next primary "]').click()
        except:
            print("-------해당 호텔 리뷰 마지막-------")
            driver.quit()
            break

C:\Users\mh\AppData\Local\Temp\ipykernel_20528\2653062227.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


Hotel d'Angleterre 시작


C:\Users\mh\AppData\Local\Temp\ipykernel_20528\2653062227.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(".//div[contains(@data-test-target, 'expand-review')]").click()
C:\Users\mh\AppData\Local\Temp\ipykernel_20528\2653062227.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  container = driver.find_elements_by_xpath("//div[@data-reviewid]")
C:\Users\mh\AppData\Local\Temp\ipykernel_20528\2653062227.py:28: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  dates = driver.find_elements_by_xpath(".//div[@class='cRVSd']")
C:\Users\mh\AppData\Local\Temp\ipykernel_20528\2653062227.py:30: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  stay_days = driver.find_elements_by_xpath(".//span[@class='teHYY _R Me S4 H3']")
C:\Users\mh\AppData\Local\Temp\ip

10 번째 page 수집중
20 번째 page 수집중
30 번째 page 수집중
40 번째 page 수집중
50 번째 page 수집중
60 번째 page 수집중
70 번째 page 수집중
80 번째 page 수집중
-------해당 호텔 리뷰 마지막-------


# dataframe으로만들기

In [30]:
#데이터 수 확인
print(len(hotel_name_list), len(date_list), len(stay_day_list), len(rating_list), len(title_list), len(review_list), len(trip_type_list),
     len(user_name_list)) #마지막이 user_name
print(len(set(hotel_name_list)), len(set(date_list)), len(set(stay_day_list)), len(set(rating_list)), len(set(title_list)), len(set(review_list)), 
      len(set(trip_type_list)), len(set(user_name_list)))

866 866 866 866 866 866 866 866
1 110 109 5 803 866 10 777


In [31]:
#크롤링 된 데이터 프레임으로 만들기
data1 = {'hotel':hotel_name_list,
        'date':date_list,
         'rating':rating_list,
        'title' : title_list,
         'review':review_list,
        'trip_type':trip_type_list,
         'user_name':user_name_list,
         'hotel_rating' : 5 #변경하기
        }
df1 = pd.DataFrame(data1)
df1

,hotel,date,rating,title,review,trip_type,user_name,hotel_rating
0,Hotel d'Angleterre,Dec 2,50,Friendly Service and Luxury in a Great Area,Copenhagen is a great city overall and I think...,None,Randal,5
1,Hotel d'Angleterre,Jan 2022,50,The best,This iconic hotel is amazing and staff outstan...,None,annaoestblom,5
2,Hotel d'Angleterre,Nov 2022,50,One of the very best hotels!,The hotel is fabulous! Beautifully decorated f...,None,Kevin,5
3,Hotel d'Angleterre,Nov 2022,50,Stylish and old-fashioned hotel - as expected,"It was exactly what we expected, stylish, old-...",None,Michael,5
4,Hotel d'Angleterre,Nov 2022,40,Mrs B,Fabulous couple of days spent in Copenhagen - ...,Trip type: Traveled with friend,Travel233564,5
...,...,...,...,...,...,...,...,...
861,Hotel d'Angleterre,May 2013,40,"Gracious welcome, fine dining in beautiful set...","Went for lunch with a friend, at d'Angleterre'...",Trip type: Traveled with friend,LTinka,5
862,Hotel d'Angleterre,May 2013,30,Drinks only!,The White Lady on Kongens Nytorv has come out ...,Trip type: Traveled with friend,alexandermeinertz,5
863,Hotel d'Angleterre,May 2013,20,Beautiful space! Poor reception...,As a current 5 star resort VIP Services employ...,Trip type: Traveled as a couple,ModernEdge,5
864,Hotel d'Angleterre,May 2013,40,Re-opened again today in style!,We had our wedding dinner in the restaurant of...,Trip type: Traveled as a couple,Bjørn,5
